In [98]:
import pandas as pd
import numpy as np
from numpy import array
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [99]:
electronics_data = pd.read_json(r'meta_electronics_17_18.json')

In [100]:
electronics_data

,description,title,image,brand,rank,main_cat,date,asin,feature,tech1,also_buy,price,also_view,tech2,category
0,"[, <b>True High Definition Sound:</b><br>With ...",Wireless Bluetooth Headphones Earbuds with Mic...,[https://images-na.ssl-images-amazon.com/image...,Enter The Arena,[>#950 in Cell Phones & Accessories (See Top 1...,Home Audio & Theater,"October 23, 2017",0132492776,[Superb Sound Quality: Plays crystal clear aud...,,[],$7.99,[],,"[Electronics, Headphones, Earbud Headphones]"
1,[A must-have for any medical professional spen...,Emergency and Critical Care Drug and Medicatio...,[https://images-na.ssl-images-amazon.com/image...,Hospital Procedures Consultants,"[>#290,628 in Office Products (See top 100), >...",Office Products,"January 16, 2018",0692056343,[The most comprehensive and reliable bedside e...,"class=""a-keyvalue prodDetTable"" role=""present...","[1929854439, B00KY1DZJ8, 128413699X, B00H5A36Y...",,"[B00H0QP0MA, B00KY1DZJ8, B01AX6NOTE, 192985443...",,"[Electronics, Accessories &amp; Supplies, Offi..."
2,[Maptech Puget Sound Chartbook is a 26-page pu...,Maptech Puget Sound Waterproof Chartbook 3rd E...,[https://images-na.ssl-images-amazon.com/image...,Maptech,[>#29 in Electronics &gt; Marine Electronics &...,GPS &amp; Navigation,"April 18, 2017",0743611349,[],"class=""a-keyvalue prodDetTable"" role=""present...","[B002A1GP0G, 0071420398, 0996979999, B0014438T...",,"[B002A1GP0G, B002A1LWYA, 0071420398, B00C34VCG...",,"[Electronics, Car &amp; Vehicle Electronics, M..."
3,[Your life will be complete when you have one ...,Shakespeare SHZP03 She is Fierce Zipper Pouch,[https://images-na.ssl-images-amazon.com/image...,Shakespeare,"[>#237,873 in Office Products (See top 100), >...",Office Products,"October 13, 2017",1609319745,"[Expandable base, Zipper with leather pull, 12...","class=""a-keyvalue prodDetTable"" role=""present...","[1571748458, 1501173243]",$16.00,[B073RVZYTY],,"[Electronics, Accessories &amp; Supplies, Audi..."
4,[The purpose of the Gleim Remote Pilot FAA Kno...,Gleim - Remote Pilot Knowledge Test,[https://images-na.ssl-images-amazon.com/image...,Gleim,"[>#94,766 in Electronics (See top 100)]",Car Electronics,"May 23, 2017",1618541110,"[Over 250 Multiple-Choice Questions, Learn and...","class=""a-keyvalue prodDetTable"" role=""present...",[],$17.80,"[1619546663, B01K0L2WOA, 1619545594, 161854191...",,"[Electronics, Car &amp; Vehicle Electronics, A..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11320,[],Elenxs LP-E8 Li-ion Battery 1800mAH for Canon ...,[https://images-na.ssl-images-amazon.com/image...,Canon,"[>#55,243 in Camera & Photo (See Top 100 in Ca...",Camera & Photo,"April 26, 2017",B01HJAHJ7U,[],,[],,[],,"[Electronics, Camera & Photo]"
11321,[<b>It's perfect for any professional and priv...,MUEEU 8x8ft Castle Backdrops Stairs Interior P...,[https://images-na.ssl-images-amazon.com/image...,MUEEU,"[>#8,632 in Camera & Photo > Camera & Photo Ac...",Camera & Photo,"March 2, 2017",B01HJAUFJE,[FASHION DESIGN: Different kinds of fashion ba...,,[],$4.38,[],,"[Electronics, Camera & Photo, Lighting & Studi..."
11322,[<b>It's perfect for any professional and priv...,MUEEU 5X7FT Antique American Flag Photography ...,[https://images-na.ssl-images-amazon.com/image...,MUEEU,"[>#3,869 in Camera & Photo > Camera & Photo Ac...",Camera & Photo,"March 3, 2017",B01HJATVCQ,[FASHION DESIGN: Different kinds of fashion ba...,,[],$1.98,[],,"[Electronics, Camera & Photo, Lighting & Studi..."
11323,[],MUEEU 5x7ft Spring Rose Flower Wallpaper Wall ...,[https://images-na.ssl-images-amazon.com/image...,MUEEU,"[>#198,027 in Camera & Photo > Camera & Photo ...",Camera & Photo,"March 30, 2017",B01HJAUDWS,[],,[],,[],,"[Electronics, Camera & Photo, Accessories]"


In [101]:
electronics_data['description']

0        [, <b>True High Definition Sound:</b><br>With ...
1        [A must-have for any medical professional spen...
2        [Maptech Puget Sound Chartbook is a 26-page pu...
3        [Your life will be complete when you have one ...
4        [The purpose of the Gleim Remote Pilot FAA Kno...
                               ...                        
11320                                                   []
11321    [<b>It's perfect for any professional and priv...
11322    [<b>It's perfect for any professional and priv...
11323                                                   []
11324    [, <b>Features</b>, <br>, <b>Specifications</b...
Name: description, Length: 11325, dtype: object

In [102]:
data = electronics_data[['asin','description','title','feature','main_cat','category']]

In [103]:
for i in range(len(data['category'])):
    if len(data['category'][i]) == 1:
        print(i)

In [104]:
data['main_cat']

0        Home Audio & Theater
1             Office Products
2        GPS &amp; Navigation
3             Office Products
4             Car Electronics
                 ...         
11320          Camera & Photo
11321          Camera & Photo
11322          Camera & Photo
11323          Camera & Photo
11324               Computers
Name: main_cat, Length: 11325, dtype: object

In [105]:
%%time
for i in range(len(data['description'])):
    data['description'][i] = ' '.join(map(str,data['description'][i])) 

D:\Anaconda\envs\tensorflow\lib\site-packages\IPython\core\magics\execution.py:1312: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exec(code, glob, local_ns)


Wall time: 3.68 s


In [106]:
%%time
import re

text = ' '.join(data['description'])
text = text.split()
freq_comm = pd.Series(text).value_counts()
rare = freq_comm[freq_comm.values == 1]

def get_clean_text(x):
    if type(x) is str:
        x = x.lower()
        x = re.sub('<[^<]+?>', ' ', x)
        x = re.sub(r'^https?:\/\/.*[\r\n]*', ' ', x, flags=re.MULTILINE)
        x = re.sub('[^A-Z a-z]+', ' ', x)
        x = ' '.join([t for t in x.split() if t not in rare])
        return x
    else:
        return x

data['description'] = data['description'].apply(lambda x: get_clean_text(x))

Wall time: 3.13 s


D:\Anaconda\envs\tensorflow\lib\site-packages\ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [66]:
text = data['description'].tolist()

In [67]:
text = [i for i in text if i] 

In [68]:
token = Tokenizer()
token.fit_on_texts(text)

In [69]:
vocab_size = len(token.word_index)+1

In [70]:
encoded_text = token.texts_to_sequences(text)

In [71]:
max_length = 16495
X = pad_sequences(encoded_text, maxlen=max_length,padding = 'post')

In [72]:
glove_vectors = dict()

In [75]:
%%time
file = open(r'glove.6B.100d.txt', encoding='utf-8')

for line in file:
    values = line.split()
    word = values[0]
    vectors = np.asarray(values[1:])
    glove_vectors[word] = vectors
file.close()

Wall time: 25.7 s


In [76]:
word_vector_matrix = np.zeros((vocab_size,100))
count = 0
for word,index in token.word_index.items():
    vector = glove_vectors.get(word)
    if vector is not None:
        word_vector_matrix[index] = vector
    else:
        print(word)
        count = count + 1

nbsp
mightyskins
sdvd
seifelden
besiva
hstnn
kpg
redtaro
eartips
elitebook
frequence
zenbook
instax
tvl
repositionable
kdl
lifebook
mirkit
sweatproof
sodial
iogear
upbright
youve
cecccrohs
probook
softbox
sdxc
bitspower
theyll
snagless
headphoness
tipss
tipsmicro
cableiphone
cablezippered
caseuser
manualcool
stickertechnical
ibeani
toogoo
comparation
zyxel
vivobook
microsdhc
rdquo
ptz
hgst
gejrio
ubatteries
aurabeam
acdream
eeebook
rmvb
eaget
fhd
gpio
cvbs
macbookpro
esddi
dvpfx
pfel
demciflex
startech
soundbar
grx
smartpass
varifocal
ilce
magsafe
softboxes
icasso
ultrastar
oneplus
miracast
supermicro
dgx
vpcf
ubiquiti
diams
gddr
channelist
onvif
gwcat
kddy
chuyi
aukey
ahnr
adlx
gmyle
atmega
proform
audifonos
xlsx
microatx
portege
wdh
designjet
thermaltake
transflash
travelstar
xpr
evga
mediapad
labelmanager
avrcp
matebook
miix
maxxaudio
wxhxd
grz
airmax
bluesound
dustproof
monoprice
mdash
lnv
engenius
pdvd
selfie
wndr
alphacool
macbookair
yiynova
irulu
bugubird
sabrent
ballhead
alurat

caldigit
fbga
minijack
trueharmony
iphoen
sdvwvbl
djx
psre
riaour
lowepro
xta
magicool
letratag
labelmaker
degf
degc
purposing
pfevex
wirerun
brilliantcolor
rbex
condusiv
idualdriver
enermax
hptx
txd
maintable
padd
protectivelayer
survice
wuala
sumitex
swdn
apeture
rarr
larr
turcom
flytouch
darller
strem
pitchblack
fuhu
kurio
bfx
rfx
comaptibility
torchiere
ecplse
asaep
abaep
cabcool
blkam
sterlibox
ultrasharp
kailo
flxm
hql
daypack
sanxiaobro
csasupply
pequipment
sjtrating
tfstoys
fbdimm
deltadrive
betadrive
lcsfk
lcsfm
kdpt
jldp
hmleaf
hwx
hxy
mediasonicusa
hign
onelink
carlcoo
sensemi
multimediacardplus
bioenno
mototrbo
beaglebone
mmic
homedics
wgnbpa
gbx
uatx
audiopipe
mixr
hotswap
waao
iprotect
fogproof
dmcfz
accusync
airprint
maileasy
powernap
playmemories
infolithium
mydigitalssd
crushproof
freeagent
twistlock
rechargable
definiton
scaning
zrx
waterplay
furutech
dolbytrue
cbnailstore
trendnets
networksa
mpgt
mpga
mpgw
mpgc
mglw
mgkm
mgty
mgkl
mgtx
storite
highlyflexible
powerfas

ltspn
ltspnlb
ltspnbk
ltspnwh
ltspndb
brightech
ltvw
ltspnclmpbk
newmacbook
roundflash
axc
riangle
obove
dropproof
kidspr
canare
tecnec
gepco
isrs
nxf
adapte
hhd
themonster
capacitorsrealtek
capsapp
dynasync
ssdsa
aerocool
featuresmodelaerocool
hgtypedimension
vacinput
ainput
hzpfcyes
mshold
msefficiency
hrsprotectionovp
sipoutput
woperation
cmcertificationsce
liquet
charer
featrue
bdsz
slimfit
highlightsallows
screensimple
adjustabledesigned
ukwhat
specificationsweight
dimensionsmount
maxsuitable
elementdigital
bluetoot
experiencepushing
zwnj
tspecifications
meterial
scrstches
interal
bbsx
phanteks
usbbt
softwarerealtek
permaproto
myeye
hdmihd
bionz
sonys
jblflip
guce
convinience
kupton
daliy
gpios
quickcharge
iqlggac
zvs
muslinhand
recordersthis
playersfeatures
interferencespecifications
domyos
optrusive
siiprinters
technologywithout
exceptiona
bsqmb
tenergy
mxjo
orbtronic
efest
imren
nitecore
intellicharger
tcgs
edrive
combatibility
suchacase
hannspree
itff
ozearbuds
ucff
aisuxr
bme

andoer
wtmgw
wgvf
devl
dddl
bolv
serenit
techcode
voltagelogic
techlnology
compitable
dimensione
plusplatinum
silverstones
setupif
highlightsinstant
ortofon
turntabledust
coverpower
supplyspecificationsspeed
drivespeed
matmain
steeleffective
mmoverhang
mmeffective
gcounterweight
gtracking
wdimensions
magicgate
dgnight
speakerd
spectrext
welovecase
lavida
sagitar
nosepiece
zenmuse
hsand
regulatorthree
divxall
mediaplaying
radiofm
xcar
xgps
xrca
carrry
lumiquest
hogore
spectrex
udqfvsr
rfidrc
mfrc
desfire
specificlly
desgin
andgt
bandamp
thium
contiuous
windowsme
mobilevision
customerservice
cesupply
routside
diamete
edtechinccustomerservice
hmwpe
strippackage
stripwarranty
sdnb
sdnp
funut
trustable
qg
tecbox
xmeye
formatdts
formatinputac
apairofshoes
sizeand
circuitreserved
sigeko
spycameracctv
dbix
dbpsk
dqpsk
condensingbr
powercharge
mulit
paad
cwfi
pxb
jiachan
shellpc
xmini
minitors
dpcp
windforce
throughtput
cingk
comfyview
colorblast
vcca
vccb
experiencethe
featuresblack
tastegame
